In [1]:
import re
import json
import numpy as np
import pandas as pd
from functools import partial

import sklearn
from sklearn.preprocessing import StandardScaler
import joblib

import lightgbm as lgb
from lightgbm import LGBMClassifier

## Own specific functions 
from functions import *

scaler = StandardScaler()

In [4]:
df_resampled = pd.read_csv(r"datasets\df_optimized.csv")

In [5]:
df_analysis(df_resampled, "df_resampled", analysis_type="header")


Analysis Header of df_resampled dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 797 columns
- Total of NaN values:			 48744
- Percentage of NaN:			 0.02 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 2.1 GB
